In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Getting AR model 

**Generate white noise:** 
White noise is a sequence of random variables drawn from normal distribution with mean zero and variance one.

**Initialize the time series with zeros:** 
Initializes time series with zeros

**Set initial values:** 
It sets initial values of the time series. Since AR-model of order 'p' requires the previous 'p' values to generate the next value, the initial 'p' values are set 
randomly with constant term 'c'

**Generate values for the AR(p) process:**
Using the formula:
**yt[i] = c + phi * yt[i - 1] + eps[i]**,
the function computes values for the autoregressive process. Each value is dependent on the previous 'p' values and white noise.


In [ ]:
def get_ar_model(c, p, n, phi, burnin=0):
    """
        Function get_ar_model generates values for an autoregressive (AR) time series
    model of order p
            Arguments:
                - c-- parameter represents the constant term of the AR model
                - p-- parameter denotes the order of autoregressive model
                - n-- number of values to generate
                - phi-- parameter represents the autoregressive coefficient
                - burnin -- number of initial values to discard

        Returns generated values for the AR model 
    """
    
    # Generate white noise
    np.random.seed(0)
    eps = np.random.normal(size=n + burnin)

    # Initialize the time series with zeros
    yt = np.zeros(n + burnin)

    # Set initial values
    for i in range(p):
        yt[i] = c + eps[i]
        
    #print("Setting initial 'p' value\n", yt, "\n" )

    # Generate values for the AR(p) process
    for i in range(p, n + burnin):
        yt[i] = c + phi * yt[i - 1] + eps[i]
        
    #print("Setting other values of time series\n",yt)
        
    return yt[burnin:]

In [ ]:
def plot_acf_pacf(data, lags=40):
    fig, ax = plt.subplots(2, figsize=(10, 6))
    sm.graphics.tsa.plot_acf(data, lags=lags, ax=ax[0], zero= False)
    sm.graphics.tsa.plot_pacf(data, lags=lags, ax=ax[1], zero= False)
    ax[0].set_title('Autocorrection Function (ACF)')
    ax[1].set_title('Partial Autocorrection Function (PACF)')
    plt.tight_layout()
    plt.show()


In [ ]:
n = 5000
yt_ar1 = get_ar_model(c=18, p=1, n=n, phi=0.6)

In [ ]:
def calculate_acf_pacf(data, lags=40):
    acf_values = sm.tsa.acf(data, nlags=lags)
    pacf_values = sm.tsa.pacf(data, nlags=lags)
    return acf_values, pacf_values

calculate_acf_pacf(yt_ar1)

In [ ]:
plot_acf_pacf(yt_ar1)

In [ ]:
phis = [-0.7, -0.8, -0.9]
for phi in phis:
    yt_ar1 = get_ar_model(c= 18, p=1, n=n, phi=phi)
    print(f"For phi = {phi}:")
    plot_acf_pacf(yt_ar1)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, kpss

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Are time series stationary or non-stationary?

***Augmented Dickey-Fuller (ADF) Test:***

> - Null Hypothesis (H0): The time series has a unit root, indicating it is non-stationary.
> - Alternative Hypothesis (H1): The time series does not have a unit root, indicating it is stationary.
> - If the p-value obtained from the test is less than a chosen significance level (typically 0.05), we reject the null hypothesis. In other words, if p-value < 0.05, we can consider the series as stationary; otherwise, we fail to reject the null hypothesis, suggesting that the series is non-stationary.
Example interpretation:

 - If adf_result[1] < 0.05, then the series is likely stationary.
 - If adf_result[1] >= 0.05, then the series is likely non-stationary.

***KPSS Test:***

> - Null Hypothesis (H0): The time series is stationary around a deterministic trend.
> - Alternative Hypothesis (H1): The time series has a unit root, indicating it is non-stationary.
> - If the p-value obtained from the test is less than a chosen significance level (typically 0.05), we reject the null hypothesis. In other words, if p-value < 0.05, we consider the series as non-stationary; otherwise, we fail to reject the null hypothesis, suggesting that the series is stationary.
Example interpretation:

 - If kpss_result[1] < 0.05, then the series is likely non-stationary.
 - If kpss_result[1] >= 0.05, then the series is likely stationary.

In [ ]:
phis = [-0.7, -0.8, -0.9, 1]

for phi in phis:
    model = get_ar_model(c=18, p=1, n=n, phi=phi)
    ljung_box = acorr_ljungbox(model, lags=[40], return_df=True)
    adf_result = adfuller(model)
    kpss_result = kpss(model)
    
    if adf_result[1] < 0.05 or kpss_result[1] > 0.05:
        print("The time series for phi =", phi, "is stationary.")
    else:
        print("The time series for phi =", phi, "is non-stationary.")

    
    

****When phi>=1 then our series is non-stationary****